In [1]:
from tqdm import tqdm
import random
import torch, gc
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, ConcatDataset
from torch.utils.data import Dataset
from torchsummary import summary

import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision

from timeit import default_timer as timer


import os
from PIL import Image
import pickle

import numpy as np
import pandas as pd

class SingleFolderDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_list = sorted(os.listdir(folder_path))
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.folder_path, self.image_list[idx])
        image = Image.open(img_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)

        return image

In [2]:
# GPU
device = torch.device("mps")

In [3]:
num_classes = 100
learning_rate = 2e-2
num_epochs = 1000
IMAGE_SIZE=256
momentum=0.9
weight_decay=1e-5
BATCH_SIZE = 200
RATIO=5
SAVE_FREQ=10
FROZEN_NUM=3

In [6]:
valid_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
])
test_dataset = SingleFolderDataset("test", transform=valid_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
test_num = list()
for file_name in test_dataset.image_list:
    test_num.append(int(file_name.split('.')[0]))
test_indices = sorted(range(len(test_num)), key=lambda k: test_num[k])

# Environment Setup

In [5]:
!pip install ultralytics


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


# Train


In [23]:
!yolo task=classify mode=train model=yolov8x-cls.pt imgsz=256 data=./ batch=256 epochs=5000 lr0=0.00002 workers=16 pretrained=True cos_lr=True dropout=0.2 val=True hsv_h=0.015 hsv_s=0.7 hsv_v=0.4 mixup=0.2 mosaic=0.8

100%|████████████████████████████████████████| 110M/110M [00:01<00:00, 76.4MB/s]
New https://pypi.org/project/ultralytics/8.0.235 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.232 🚀 Python-3.8.10 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24217MiB)
engine/trainer: task=classify, mode=train, model=yolov8x-cls.pt, data=./, epochs=5000, time=None, patience=50, batch=256, imgsz=256, save=True, save_period=-1, cache=False, device=None, workers=16, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, cla

In [ ]:
from ultralytics import YOLO
import os

# Load a model
model = YOLO("yolov8x-cls.yaml")  # build a new model from scratch
model = YOLO(os.path.join("runs", "classify", "train7", "weights", "best.pt") ) # load a pretrained model (recommended for training)
model = model.to(device)



                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   7375360  ultralytics.nn.modules.conv.Conv             [640

  8                  -1  3  27865600  ultralytics.nn.modules.block.C2f             [1280, 1280, 3, True]         
  9                  -1  1   2921960  ultralytics.nn.modules.head.Classify         [1280, 1000]                  
YOLOv8x-cls summary: 183 layers, 57422840 parameters, 57422840 gradients, 155.3 GFLOPs


# Inference Probability Vector

In [ ]:
label_pred = list()
tensorList = []
with torch.no_grad():
    for images in (test_loader):
        images = images.to(device)
        outputs = model(images)
        index = outputs[0].probs.top5
        prob = outputs[0].probs.top5conf
        tensor = torch.zeros(1, 100)
        for i in range(5):
            tensor[0][index[i]] = prob[i]
        tensorList.append(tensor)
resultTensor = torch.cat(tensorList, dim=0)
with open(os.path.join('yolov8-prob-vector.pickle'), 'wb') as file:
    pickle.dump(resultTensor, file)


0: 256x256 polo 1.00, horse racing 0.00, barell racing 0.00, harness racing 0.00, horse jumping 0.00, 5.5ms
Speed: 0.1ms preprocess, 5.5ms inference, 20.2ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 sky surfing 0.99, snow boarding 0.00, bungee jumping 0.00, skydiving 0.00, high jump 0.00, 8.8ms
Speed: 0.0ms preprocess, 8.8ms inference, 0.3ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 nascar racing 1.00, formula 1 racing 0.00, football 0.00, hockey 0.00, sidecar racing 0.00, 6.0ms
Speed: 0.0ms preprocess, 6.0ms inference, 0.2ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 curling 0.99, speed skating 0.01, judo 0.00, snow boarding 0.00, luge 0.00, 5.4ms
Speed: 0.0ms preprocess, 5.4ms inference, 0.2ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 horse racing 0.86, barell racing 0.05, polo 0.05, horse jumping 0.02, harness racing 0.01, 6.2ms
Speed: 0.0ms preprocess, 6.2ms inference, 0.3ms postprocess per image at shape (1, 3

KeyboardInterrupt: 

# Test Accuracy

In [7]:
label_true = pd.read_csv('Test_label.csv')
with open(os.path.join("results", "yolov8-prob-vector.pickle"), 'rb') as file:
    yoloV = pickle.load(file)

name = 'yoloV8'
yoloV = yoloV.cpu().numpy()[test_indices,:]
def acc_calculate(name,
                  prob,
                  label_true):
    prob = prob_list[i]
    prob = np.array(prob)
    prob_label = pd.DataFrame(prob.argmax(axis=1), columns=['Category'])
    acc = (sum(prob_label['Category'] == label_true['Category'])/500)
    return_dict = {'model' : name, 'acc' : list(acc)}
    return pd.DataFrame(return_dict).sort_values(by='acc')

accuracy = acc_calculate(name, yoloV, )
print(accuracy)


FileNotFoundError: [Errno 2] No such file or directory: 'results/yolov8-prob-vector.pickle'